Loading the dataset


In [ ]:
from google.colab import files
files.upload() #upload your kaggle key


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d grikomsn/amazon-cell-phones-reviews
!unzip amazon-cell-phones-reviews.zip


Dataset URL: https://www.kaggle.com/datasets/grikomsn/amazon-cell-phones-reviews
License(s): CC0-1.0
 57% 5.00M/8.84M [00:00<00:00, 20.1MB/s]
100% 8.84M/8.84M [00:00<00:00, 32.0MB/s]
Archive:  amazon-cell-phones-reviews.zip
  inflating: 20191226-items.csv      
  inflating: 20191226-reviews.csv    


import necessary libraries

In [27]:
# Install necessary libraries
!pip install transformers
!pip install sentence-transformers

# Import libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from transformers import pipeline
from tqdm import tqdm
import torch

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
items_df = pd.read_csv('20191226-items.csv')
reviews_df = pd.read_csv('20191226-reviews.csv')

# Join datasets on 'asin'
df = pd.merge(reviews_df, items_df, on='asin', how='left')
df.head()


,asin,name,rating_x,date,verified,title_x,body,helpfulVotes,brand,title_y,url,image,rating_y,reviewUrl,totalReviews,price,originalPrice
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.0,0.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.0,0.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.0,0.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.0,0.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.0,0.0


In [29]:
# Sample 10% of the dataset
df_sampled = df.sample(frac=0.1, random_state=42)
df_sampled.shape


(6799, 17)

In [31]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()  # Lowercase
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
        tokens = word_tokenize(text)  # Tokenize
        tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
        return ' '.join(tokens)
    else:
        return ''  # Return empty string if text is not a string

df_sampled['cleaned_review'] = df_sampled['body'].apply(preprocess_text)
df_sampled.head()


,asin,name,rating_x,date,verified,title_x,body,helpfulVotes,brand,title_y,url,image,rating_y,reviewUrl,totalReviews,price,originalPrice,cleaned_review
29219,B01N9XOXCK,magnum688,5,"February 22, 2018",True,Best phone ever,Nothing but flying star's for me here I had a ...,NaN,Samsung,Samsung Galaxy S7 Edge G935V 32GB Gold - Veriz...,https://www.amazon.com/Samsung-Galaxy-Edge-G93...,https://m.media-amazon.com/images/I/41K7nb5aDB...,3.8,https://www.amazon.com/product-reviews/B01N9XOXCK,606,219.97,0.0,nothing flying stars ss6 years dosent work got...
57965,B07NZVM3RN,ian herrington,5,"July 25, 2019",True,S 10 e an amazeing phone,Really awesome phone i love it,1.0,Samsung,Samsung Galaxy S10e 128GB+6GB RAM SM-G970 Dual...,https://www.amazon.com/Samsung-S10e-Unlocked-S...,https://m.media-amazon.com/images/I/51BateZ5iq...,4.5,https://www.amazon.com/product-reviews/B07NZVM3RN,213,541.98,0.0,really awesome phone love
48131,B07CMBB6PH,Martin,5,"December 17, 2018",True,Awesome phone and awesome price,Awesome phone !!! Although it’s a lite version...,NaN,HUAWEI,"HUAWEI P20 Lite (32GB + 4GB RAM) 5.84"" FHD+ Di...",https://www.amazon.com/Huawei-ANE-LX3-Factory-...,https://m.media-amazon.com/images/I/819xBtcnz4...,4.3,https://www.amazon.com/product-reviews/B07CMBB6PH,558,229.99,0.0,awesome phone although lite version phone amaz...
15033,B00TRLXO6U,Amazon Customer,2,"June 27, 2016",True,only works with Verizon as a service provider.,Great camera. Window is amazing. Only compatib...,NaN,Nokia,Nokia Lumia 928 32GB Unlocked GSM 4G LTE Windo...,https://www.amazon.com/Nokia-Unlocked-Windows-...,https://m.media-amazon.com/images/I/81CFhqN240...,3.8,https://www.amazon.com/product-reviews/B00TRLXO6U,112,0.00,0.0,great camera window amazing compatible verizon...
66979,B07X5VF1FM,luis francisco espinoza,5,"December 10, 2019",True,Super genial el teléfono,Un gran teléfono y el color es genial,NaN,Xiaomi,"Xiaomi Mi 9T Pro (128GB, 6GB RAM) 6.39"" Displa...",https://www.amazon.com/Xiaomi-Display-Snapdrag...,https://m.media-amazon.com/images/I/61wqISkeqC...,4.3,https://www.amazon.com/product-reviews/B07X5VF1FM,44,379.00,0.0,un gran telfono el color es genial


In [41]:
# Ensure the model runs on GPU if available
device = 0 if torch.cuda.is_available() else -1
absa = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)

aspects = ["battery", "screen", "performance", "camera", "design"]

def extract_aspect_sentiments(review):
    sentiments = {}
    max_len = 512
    for aspect in aspects:
        text = (review + " " + aspect)[:max_len]  # Truncate to max_len
        sentiment = absa(text)
        sentiments[aspect] = sentiment[0]['label']
    return sentiments

# Apply in batches to avoid long processing times
batch_size = 10  # Adjust batch size as needed
aspect_sentiments_list = []

for i in tqdm(range(0, len(df_sampled), batch_size)):
    end = i + batch_size if i + batch_size < len(df_sampled) else len(df_sampled)
    batch_reviews = df_sampled['cleaned_review'].iloc[i:end].apply(extract_aspect_sentiments)
    aspect_sentiments_list.extend(batch_reviews)

df_sampled['aspect_sentiments'] = aspect_sentiments_list
df_sampled.head()
df_sampled.tail()


100%|██████████| 680/680 [04:51<00:00,  2.33it/s]


,asin,name,rating_x,date,verified,title_x,body,helpfulVotes,brand,title_y,...,image,rating_y,reviewUrl,totalReviews,price,originalPrice,cleaned_review,aspect_sentiments,aspect_scores,label
62784,B07R4PP7FF,Steven W Perkins,5,"August 29, 2019",True,This is the best phone that Americans have nev...,"I've had this phone for several months, and I ...",8.0,Xiaomi,Xiaomi Mi 9 64GB + 6GB RAM - 48MP Ultra High R...,...,https://m.media-amazon.com/images/I/51OANpTqi8...,4.5,https://www.amazon.com/product-reviews/B07R4PP7FF,84,334.90,0.00,ive phone several months still rate 5 stars to...,"{'battery': '4 stars', 'screen': '4 stars', 'p...","{'battery': 4, 'screen': 4, 'performance': 4, ...",True
41511,B076XLLCQC,Shelly W.,5,"July 19, 2019",True,"My Amazon Prime, you've done it again!",I purchased this phone used. It came in almost...,NaN,Samsung,Samsung Galaxy Note 8 N950 Factory Unlocked Ph...,...,https://m.media-amazon.com/images/I/615Y7qMu7l...,4.0,https://www.amazon.com/product-reviews/B076XLLCQC,891,330.95,0.00,purchased phone used came almost perfect condi...,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",True
22776,B01DZ2CS4U,Charity,5,"February 7, 2019",True,Great product,"Great phone, great condition. Was not able to ...",NaN,Samsung,Samsung Galaxy S7 32GB G930V Unlocked - Black,...,https://m.media-amazon.com/images/I/51i3dF4frh...,3.2,https://www.amazon.com/product-reviews/B01DZ2CS4U,250,275.99,0.00,great phone great condition able use hotspot v...,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",True
57520,B07NPRQYP1,mike wolf,5,"April 23, 2019",True,Good value for a good phone just missing some ...,I have to say the only thing I don't like so f...,1.0,Motorola,Moto G7 Power - Unlocked - 64 GB - Marine Blue...,...,https://m.media-amazon.com/images/I/71nplhIIYj...,4.0,https://www.amazon.com/product-reviews/B07NPRQYP1,455,165.00,299.99,say thing dont like far getting cast tv use mi...,"{'battery': '1 star', 'screen': '1 star', 'per...","{'battery': 1, 'screen': 1, 'performance': 1, ...",True
36849,B072N3GKSM,R. Taylor,1,"January 21, 2019",True,"Looks all new, except battery","I bought renewed from Amazon, not a 3rd party....",3.0,Google,"Google Pixel GSM Unlocked (Renewed) (32GB, Gray)",...,https://m.media-amazon.com/images/I/81o50fc16k...,2.9,https://www.amazon.com/product-reviews/B072N3GKSM,319,109.95,0.00,bought renewed amazon 3rd party phone looks fl...,"{'battery': '1 star', 'screen': '1 star', 'per...","{'battery': 1, 'screen': 1, 'performance': 1, ...",False


In [42]:
df_sampled.head()

,asin,name,rating_x,date,verified,title_x,body,helpfulVotes,brand,title_y,...,image,rating_y,reviewUrl,totalReviews,price,originalPrice,cleaned_review,aspect_sentiments,aspect_scores,label
29219,B01N9XOXCK,magnum688,5,"February 22, 2018",True,Best phone ever,Nothing but flying star's for me here I had a ...,NaN,Samsung,Samsung Galaxy S7 Edge G935V 32GB Gold - Veriz...,...,https://m.media-amazon.com/images/I/41K7nb5aDB...,3.8,https://www.amazon.com/product-reviews/B01N9XOXCK,606,219.97,0.0,nothing flying stars ss6 years dosent work got...,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",True
57965,B07NZVM3RN,ian herrington,5,"July 25, 2019",True,S 10 e an amazeing phone,Really awesome phone i love it,1.0,Samsung,Samsung Galaxy S10e 128GB+6GB RAM SM-G970 Dual...,...,https://m.media-amazon.com/images/I/51BateZ5iq...,4.5,https://www.amazon.com/product-reviews/B07NZVM3RN,213,541.98,0.0,really awesome phone love,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",True
48131,B07CMBB6PH,Martin,5,"December 17, 2018",True,Awesome phone and awesome price,Awesome phone !!! Although it’s a lite version...,NaN,HUAWEI,"HUAWEI P20 Lite (32GB + 4GB RAM) 5.84"" FHD+ Di...",...,https://m.media-amazon.com/images/I/819xBtcnz4...,4.3,https://www.amazon.com/product-reviews/B07CMBB6PH,558,229.99,0.0,awesome phone although lite version phone amaz...,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",True
15033,B00TRLXO6U,Amazon Customer,2,"June 27, 2016",True,only works with Verizon as a service provider.,Great camera. Window is amazing. Only compatib...,NaN,Nokia,Nokia Lumia 928 32GB Unlocked GSM 4G LTE Windo...,...,https://m.media-amazon.com/images/I/81CFhqN240...,3.8,https://www.amazon.com/product-reviews/B00TRLXO6U,112,0.00,0.0,great camera window amazing compatible verizon...,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",False
66979,B07X5VF1FM,luis francisco espinoza,5,"December 10, 2019",True,Super genial el teléfono,Un gran teléfono y el color es genial,NaN,Xiaomi,"Xiaomi Mi 9T Pro (128GB, 6GB RAM) 6.39"" Displa...",...,https://m.media-amazon.com/images/I/61wqISkeqC...,4.3,https://www.amazon.com/product-reviews/B07X5VF1FM,44,379.00,0.0,un gran telfono el color es genial,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",True


In [43]:
def sentiment_to_score(sentiment):
    mapping = {"1 star": 1, "2 stars": 2, "3 stars": 3, "4 stars": 4, "5 stars": 5}
    return mapping[sentiment]

df_sampled['aspect_scores'] = df_sampled['aspect_sentiments'].apply(lambda x: {k: sentiment_to_score(v) for k, v in x.items()})
df_sampled.head()


,asin,name,rating_x,date,verified,title_x,body,helpfulVotes,brand,title_y,...,image,rating_y,reviewUrl,totalReviews,price,originalPrice,cleaned_review,aspect_sentiments,aspect_scores,label
29219,B01N9XOXCK,magnum688,5,"February 22, 2018",True,Best phone ever,Nothing but flying star's for me here I had a ...,NaN,Samsung,Samsung Galaxy S7 Edge G935V 32GB Gold - Veriz...,...,https://m.media-amazon.com/images/I/41K7nb5aDB...,3.8,https://www.amazon.com/product-reviews/B01N9XOXCK,606,219.97,0.0,nothing flying stars ss6 years dosent work got...,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",True
57965,B07NZVM3RN,ian herrington,5,"July 25, 2019",True,S 10 e an amazeing phone,Really awesome phone i love it,1.0,Samsung,Samsung Galaxy S10e 128GB+6GB RAM SM-G970 Dual...,...,https://m.media-amazon.com/images/I/51BateZ5iq...,4.5,https://www.amazon.com/product-reviews/B07NZVM3RN,213,541.98,0.0,really awesome phone love,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",True
48131,B07CMBB6PH,Martin,5,"December 17, 2018",True,Awesome phone and awesome price,Awesome phone !!! Although it’s a lite version...,NaN,HUAWEI,"HUAWEI P20 Lite (32GB + 4GB RAM) 5.84"" FHD+ Di...",...,https://m.media-amazon.com/images/I/819xBtcnz4...,4.3,https://www.amazon.com/product-reviews/B07CMBB6PH,558,229.99,0.0,awesome phone although lite version phone amaz...,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",True
15033,B00TRLXO6U,Amazon Customer,2,"June 27, 2016",True,only works with Verizon as a service provider.,Great camera. Window is amazing. Only compatib...,NaN,Nokia,Nokia Lumia 928 32GB Unlocked GSM 4G LTE Windo...,...,https://m.media-amazon.com/images/I/81CFhqN240...,3.8,https://www.amazon.com/product-reviews/B00TRLXO6U,112,0.00,0.0,great camera window amazing compatible verizon...,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",False
66979,B07X5VF1FM,luis francisco espinoza,5,"December 10, 2019",True,Super genial el teléfono,Un gran teléfono y el color es genial,NaN,Xiaomi,"Xiaomi Mi 9T Pro (128GB, 6GB RAM) 6.39"" Displa...",...,https://m.media-amazon.com/images/I/61wqISkeqC...,4.3,https://www.amazon.com/product-reviews/B07X5VF1FM,44,379.00,0.0,un gran telfono el color es genial,"{'battery': '5 stars', 'screen': '5 stars', 'p...","{'battery': 5, 'screen': 5, 'performance': 5, ...",True


In [44]:
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Creating dummy features and labels for illustration purposes
df_sampled['label'] = df_sampled['rating_x'] > 3

X = df_sampled['aspect_scores'].apply(lambda x: list(x.values())).tolist()
X = np.array(X)
y = df_sampled['label'].astype(int)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Calculate Mean Average Precision (MAP)
map_score = average_precision_score(y_test, y_prob)
print(f"Mean Average Precision (MAP): {map_score}")


Mean Average Precision (MAP): 0.909531496853987
